In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import sys
sys.path.insert(0,"..")

# Import packages and setup base classes

In [41]:
import torch
import torch.optim as optim
from types import SimpleNamespace as Namespace

In [42]:
from utils.training_loop import TrainingLoop
from custom_datasets.messidor import Messidor
from custom_datasets.bcn_20000 import BCN20000
from utils.construct_model import LoadVICRegModel
from utils.training_loop import TrainingLoop


In [43]:
gpu = torch.cuda.current_device()
torch.cuda.set_device(gpu)
torch.backends.cudnn.benchmark = True

## namespace mock

In [44]:
args = Namespace(
    batch_size=16,
    num_workers=4,
    gpu=None,
    weight_decay=0.0001,
    epochs=40,
    weights="finetune",
    print_freq=50,
    exp_dir="./test_expr/",
    lr_backbone=3e-3,
    lr_head=0.03,
)


In [45]:
dataset = BCN20000(
    batch_size=64,
    num_workers=16,
    gpu=gpu,
)

# Infra

## dataloaders

In [46]:
train_loader = dataset.get_dataloader(split="train")
val_loader = dataset.get_dataloader(split="valid")

length of dataset train: 20265
length of dataset valid: 5066


## model

In [47]:
model = LoadVICRegModel(arch="resnet50")
model.load_pretrained_weights(pretrained_path="../VICReg_RadImageNet/resnet50.pth")
model.modify_head(num_classes=dataset.num_classes)
model = model.produce_model()
model.cuda(gpu)
param_groups = [dict(params=model[-1].parameters(), lr=args.lr_head)]
if args.weights == "finetune":
    param_groups.append(dict(params=model[:-1].parameters(), lr=args.lr_backbone))

Loading local VICReg ResNet50 arch Model


In [48]:
## optimizer
optimizer = optim.Adam(param_groups, 0, weight_decay=args.weight_decay)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, args.epochs)


# training loop


In [49]:
training_loop = TrainingLoop(
    model,
    optimizer,
    scheduler,
    train_loader,
    val_loader,
    dataset,
    args,
    gpu=gpu,
    stats_file=None,
    multi_label=False,
)

In [50]:
training_loop.train(
    start_epoch=0, num_epochs=args.epochs)

2025-02-05 06:07 [info     ] Beginning training            


2025-02-05 06:07 [info     ] {"epoch": 0, "step": 0, "lr_backbone": 0.003, "lr_head": 0.03, "loss": 1.6263223886489868, "time": 1}
2025-02-05 06:07 [info     ] {"epoch": 0, "step": 0, "lr_backbone": 0.003, "lr_head": 0.03, "loss": 1.6263223886489868, "time": 1}
2025-02-05 06:07 [info     ] {"epoch": 0, "step": 50, "lr_backbone": 0.003, "lr_head": 0.03, "loss": 1.1835793256759644, "time": 10}
2025-02-05 06:07 [info     ] {"epoch": 0, "step": 50, "lr_backbone": 0.003, "lr_head": 0.03, "loss": 1.1835793256759644, "time": 10}
2025-02-05 06:07 [info     ] {"epoch": 0, "step": 100, "lr_backbone": 0.003, "lr_head": 0.03, "loss": 1.0321677923202515, "time": 19}
2025-02-05 06:07 [info     ] {"epoch": 0, "step": 100, "lr_backbone": 0.003, "lr_head": 0.03, "loss": 1.0321677923202515, "time": 19}
2025-02-05 06:08 [info     ] {"epoch": 0, "step": 150, "lr_backbone": 0.003, "lr_head": 0.03, "loss": 1.3576756715774536, "time": 28}
2025-02-05 06:08 [info     ] {"epoch": 0, "step": 150, "lr_backbone": 